In [21]:
#importing the relevant libraries
import tqdm
import pandas as pd
import io
import requests
import PyPDF2
import json
from bs4 import BeautifulSoup

In [25]:
ids = pd.read_csv('number_id_list_2.csv')
id_item = ids['number_id']

#pointing to the relevant url, steps are: 1) getting a response from the page 2) passing the response to a BeautifoulSoup
#object 3) finding the tag with the relevant, final link 4) get that link (finding the tag exploring the html structure)
#5) get a response from that link (which is the pdf) and passing its content as BytesIO object to the PDF reader
for i in tqdm.tqdm(list(id_item[2527:2636])):
    url = 'https://www.universitaly.it/index.php/scheda/sua/' + str(i)
    response = requests.get(url)
    page_html = BeautifulSoup(response.text, "html.parser")
    
    #this is the relevant tag where the ref to pdf is stored
    pdf_tag = page_html.find("a",{"style":"font-size: 10pt;text-decoration: none", "target":"_blank"}).get("href")
    
    #we store it in the response variable, ready to be written somewhere
    response_pdf = requests.get(pdf_tag)
    
    
    #passing the content as BytesIO object to the PDF reader
    pdf_file = PyPDF2.PdfFileReader(io.BytesIO(response_pdf.content), strict=False)
    
    #we use a PyPDF2 function to get the page number, the page itself and the text inside this page. With these ingredients
    #we loop through everything and append the text
    
    text = ''
    for item in range(pdf_file.getNumPages()):
        text += pdf_file.getPage(item).extractText()

    # Convert the text to a dictionary
    data = {'text': text}

    # Convert the dictionary to JSON format
    json_data = json.dumps(data)

    # Write the JSON data to a file
    with open('./Full_Download_PDF_in_JSON/file_downloaded_batch_2/' +str(i) + '.json', 'w') as json_file:
        json_file.write(json_data)


100%|█████████████████████████████████████████| 109/109 [17:36<00:00,  9.69s/it]


In [26]:
id_item[id_item == i]

2635    56425
Name: number_id, dtype: int64

In [18]:
len(id_item)

2686